In [ ]:
Following https://juliasymbolics.github.io/Metatheory.jl/dev/

In [1]:
using Pkg; Pkg.add(url="https://github.com/JuliaSymbolics/Metatheory.jl")

     Cloning git-repo `https://github.com/JuliaSymbolics/Metatheory.jl`


    Updating git-repo `https://github.com/JuliaSymbolics/Metatheory.jl`


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed AutoHashEquals ─ v0.2.0
   Installed TermInterface ── v0.3.3


    Updating `/workspaces/julia-playground/GARules.jl/Project.toml`
  [e9d8d322] + Metatheory v2.0.1 `https://github.com/JuliaSymbolics/Metatheory.jl#master`
    Updating `/workspaces/julia-playground/GARules.jl/Manifest.toml`
  [15f4f7f2] + AutoHashEquals v0.2.0
  [e9d8d322] + Metatheory v2.0.1 `https://github.com/JuliaSymbolics/Metatheory.jl#master`
  [8ea1fca8] + TermInterface v0.3.3


Precompiling project...


  ✓ AutoHashEquals
  ✓ TermInterface


  ✓ Metatheory


  ✓ GARules
  4 dependencies successfully precompiled in 4 seconds. 57 already precompiled.


In [55]:
using Pkg; Pkg.add("Metatheory")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


    Updating `/workspaces/julia-playground/GARules.jl/Project.toml`
  [e9d8d322] ~ Metatheory v2.0.1 `https://github.com/JuliaSymbolics/Metatheory.jl#master` ⇒ v2.0.1
    Updating `/workspaces/julia-playground/GARules.jl/Manifest.toml`
  [e9d8d322] ~ Metatheory v2.0.1 `https://github.com/JuliaSymbolics/Metatheory.jl#master` ⇒ v2.0.1


In [56]:
using Metatheory

In [57]:
r1 = @rule sin(2(~x)) --> 2sin(~x)*cos(~x)

sin(2 * ~x) --> (2 * sin(~x)) * cos(~x)

In [58]:
expr = :(sin(2z))

:(sin(2z))

In [59]:
r1(expr)

:((2 * sin(z)) * cos(z))

In [60]:
r1(:(sin(3z))) === nothing

true

In [61]:
r1(:(sin(2*(w-z))))

:((2 * sin(w - z)) * cos(w - z))

In [62]:
r1(:(sin(2*(w+z)*(α+β)))) === nothing

true

In [63]:
r2 = @rule sin(~x + ~y) --> sin(~x)*cos(~y) + cos(~x)*sin(~y)

sin(~x + ~y) --> sin(~x) * cos(~y) + cos(~x) * sin(~y)

In [64]:
@rule(+(~xs...) => xs)(:(x + y + z))

3-element view(::Vector{Any}, 1:3) with eltype Any:
 :x
 :y
 :z

In [65]:
r3 = @rule *(~ys...)^~x => :((*)($(map(y-> :($y^$x), ys)...)))

(*)(~ys...) ^ ~x => Core._expr(:call, :*, map((y->begin
                    Core._expr(:call, :^, y, x)
                end), ys)...)

In [66]:
r3(:((w*w*α*β)^2))

:(w ^ 2 * w ^ 2 * α ^ 2 * β ^ 2)

In [67]:
r = @rule +(~x, ~y::(ys->iseven(length(ys)))...) => "odd terms";

In [68]:
@show r(:(a + b + c + d))
@show r(:(b + c + d))
@show r(:(b + c + b))
@show r(:(a + b))

r($(Expr(:quote, :(a + b + c + d)))) = nothing
r($(Expr(:quote, :(b + c + d)))) = "odd terms"
r($(Expr(:quote, :(b + c + b)))) = "odd terms"
r($(Expr(:quote, :(a + b)))) = nothing


In [69]:
@slots x y @rule sin(x + y) => sin(x)*cos(y) + cos(x)*sin(y)

sin(~x + ~y) => sin(x) * cos(y) + cos(x) * sin(y)

In [70]:
@slots xs @rule(+(~xs...) => xs);

In [71]:
@rule x y sin(x + y) => sin(x)*cos(y) + cos(x)*sin(y)

sin(~x + ~y) => sin(x) * cos(y) + cos(x) * sin(y)

In [72]:
t = @theory x y z begin 
    x * (y + z) --> (x * y) + (x * z)
    x + y       ==  (y + x)
    #...
end

2-element Vector{SymbolicRule}:
 ~x * (~y + ~z) --> ~x * ~y + ~x * ~z
 ~x + ~y == ~y + ~x

In [73]:
using Metatheory.Library

In [183]:
comm_monoid = @commutative_monoid (*) 1
comm_group = @theory a b c begin
    a + 0 --> a
    a + b --> b + a
    a + inv(a) --> 0 # inverse
    a + (b + c) --> (a + b) + c
end
distrib = @theory a b c begin
    a * (b + c) --> (a * b) + (a * c)
end
t = comm_monoid ∪ comm_group ∪ distrib

9-element Vector{RewriteRule}:
 ~a * ~b --> ~b * ~a
 (~a * ~b) * ~c --> ~a * (~b * ~c)
 ~a * (~b * ~c) --> (~a * ~b) * ~c
 1 * ~a --> ~a
 ~a + 0 --> ~a
 ~a + ~b --> ~b + ~a
 ~a + inv(~a) --> 0
 ~a + (~b + ~c) --> (~a + ~b) + ~c
 ~a * (~b + ~c) --> ~a * ~b + ~a * ~c

In [75]:
rsin = @rule x sin(2x) == 2sin(x)*cos(x)

sin(2 * ~x) == (2 * sin(~x)) * cos(~x)

In [77]:
comm_monoid

4-element Vector{RewriteRule}:
 ~a * ~b --> ~b * ~a
 (~a * ~b) * ~c --> ~a * (~b * ~c)
 ~a * (~b * ~c) --> (~a * ~b) * ~c
 1 * ~a --> ~a

In [78]:
t2 = @theory a b c begin
    a * b == b * a
    a * 1 == a
    a * (b * c) == (a * b) * c
end

3-element Vector{EqualityRule}:
 ~a * ~b == ~b * ~a
 ~a * 1 == ~a
 ~a * (~b * ~c) == (~a * ~b) * ~c

In [152]:
Metatheory.resetbuffers!(Metatheory.DEFAULT_BUFFER_SIZE)

CircularDeque{Tuple{Int64, Int64}}([])

In [153]:
g = EGraph(:((a * b) * (1 * (b + c))))

EGraph(IntDisjointSet([-1, -1, -1, -1, -1, -1, -1, -1], Base.RefValue{Bool}(true)), Dict{Int64, EClass}(5 => EClass 5 ([c], NamedTuple()), 4 => EClass 4 ([1], NamedTuple()), 6 => EClass 6 ([ENode(call, +, Expr, [2, 5])], NamedTuple()), 7 => EClass 7 ([ENode(call, *, Expr, [4, 6])], NamedTuple()), 2 => EClass 2 ([b], NamedTuple()), 8 => EClass 8 ([ENode(call, *, Expr, [3, 7])], NamedTuple()), 3 => EClass 3 ([ENode(call, *, Expr, [1, 2])], NamedTuple()), 1 => EClass 1 ([a], NamedTuple())), Dict{AbstractENode, Int64}(ENode(call, *, Expr, [4, 6]) => 7, a => 1, c => 5, ENode(call, *, Expr, [1, 2]) => 3, ENode(call, +, Expr, [2, 5]) => 6, ENode(call, *, Expr, [3, 7]) => 8, b => 2, 1 => 4), Int64[], 8, Dict{Union{Function, Symbol}, Union{Function, Symbol}}(), Dict{Any, Vector{Int64}}(:a => [1], :b => [2], :+ => [6], :c => [5], :* => [3, 7, 8], 1 => [4]), Expr, Dict{Tuple{Any, Int64}, Type}(), 8, 0)

In [154]:
report = saturate!(g, t2)

SaturationReport
	Stop Reason: saturated
	Iterations: 4
	EGraph Size: 9 eclasses, 72 nodes
 ────────────────────────────────────────────────────────────────────
                            Time                    Allocations      
                   ───────────────────────   ────────────────────────
 Tot / % measured:     3.24ms /  95.0%            829KiB /  99.2%    

 Section   ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────
 Apply          4   1.88ms   61.0%   470μs    439KiB   53.3%   110KiB
 Search         4    937μs   30.4%   234μs    325KiB   39.5%  81.2KiB
   3            4    656μs   21.3%   164μs    208KiB   25.4%  52.1KiB
   1            4    139μs    4.5%  34.7μs   62.9KiB    7.6%  15.7KiB
   2            4    124μs    4.0%  31.1μs   50.1KiB    6.1%  12.5KiB
 Rebuild        4    263μs    8.6%  65.9μs   59.1KiB    7.2%  14.8KiB
 ───────────────────────────────────────────────────────────────────

In [155]:
extract!(g, astsize)

:((b + c) * (b * a))

In [156]:
rsin(:(sin(2 * x)))

Metatheory.Rules.RuleRewriteError: Failed to apply rule sin(2 * ~x) == (2 * sin(~x)) * cos(~x) on expression :(sin(2x))

In [177]:
g = EGraph(:(2sin(x)*cos(x)))

EGraph(IntDisjointSet([-1, -1, -1, -1, -1, -1], Base.RefValue{Bool}(true)), Dict{Int64, EClass}(5 => EClass 5 ([ENode(call, cos, Expr, [2])], NamedTuple()), 4 => EClass 4 ([ENode(call, *, Expr, [1, 3])], NamedTuple()), 6 => EClass 6 ([ENode(call, *, Expr, [4, 5])], NamedTuple()), 2 => EClass 2 ([x], NamedTuple()), 3 => EClass 3 ([ENode(call, sin, Expr, [2])], NamedTuple()), 1 => EClass 1 ([2], NamedTuple())), Dict{AbstractENode, Int64}(x => 2, 2 => 1, ENode(call, *, Expr, [1, 3]) => 4, ENode(call, cos, Expr, [2]) => 5, ENode(call, *, Expr, [4, 5]) => 6, ENode(call, sin, Expr, [2]) => 3), Int64[], 6, Dict{Union{Function, Symbol}, Union{Function, Symbol}}(), Dict{Any, Vector{Int64}}(2 => [1], :cos => [5], :* => [4, 6], :sin => [3], :x => [2]), Expr, Dict{Tuple{Any, Int64}, Type}(), 6, 0)

In [178]:
report = saturate!(g, [rsin])

SaturationReport
	Stop Reason: saturated
	Iterations: 2
	EGraph Size: 7 eclasses, 8 nodes
 ────────────────────────────────────────────────────────────────────
                            Time                    Allocations      
                   ───────────────────────   ────────────────────────
 Tot / % measured:      397μs /  63.7%           24.6KiB /  73.0%    

 Section   ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────
 Search         2    121μs   47.9%  60.6μs   11.2KiB   62.2%  5.58KiB
   1            2    114μs   44.9%  56.8μs   10.2KiB   57.1%  5.12KiB
 Apply          2   98.7μs   39.0%  49.4μs   5.08KiB   28.3%  2.54KiB
 Rebuild        2   33.0μs   13.1%  16.5μs   1.70KiB    9.5%     872B
 ────────────────────────────────────────────────────────────────────


In [179]:
extract!(g, astsize)

:(sin(2x))

In [181]:
comm_group

4-element Vector{RewriteRule}:
 ~a + 0 --> ~a
 ~a + ~b --> ~b + ~a
 ~a + inv(~a) --> 0
 ~a + (~b + ~c) --> (~a + ~b) + ~c

In [157]:
@areequal comm_group (x+y)*(a+b) ((a*(x+y))+b*(x+y)) ((x*(a+b))+y*(a+b)) 

false

In [184]:
t

9-element Vector{RewriteRule}:
 ~a * ~b --> ~b * ~a
 (~a * ~b) * ~c --> ~a * (~b * ~c)
 ~a * (~b * ~c) --> (~a * ~b) * ~c
 1 * ~a --> ~a
 ~a + 0 --> ~a
 ~a + ~b --> ~b + ~a
 ~a + inv(~a) --> 0
 ~a + (~b + ~c) --> (~a + ~b) + ~c
 ~a * (~b + ~c) --> ~a * ~b + ~a * ~c

In [185]:
@areequal t a * b * c b * (a * c)

true

In [186]:
@areequal t (x+y)*(a+b) ((a*(x+y))+b*(x+y)) ((x*(a+b))+y*(a+b)) 

true

In [187]:
t3 = @theory a b c begin
    a + 0 --> a
    a + b --> b + a
    a + inv(a) --> 0 # inverse
    a + (b + c) --> (a + b) + c
	a * (b + c) --> (a * b) + (a * c)
	(a * b) + (a * c) --> a * (b + c)
	a * a --> a^2
	a --> a^1
	a^b * a^c --> a^(b+c)
	log(a^b) --> b * log(a)
	log(a * b) --> log(a) + log(b)
	log(1) --> 0
	log(:e) --> 1
	:e^(log(a)) --> a
	a::Number + b::Number => a + b
	a::Number * b::Number => a * b
end
t3 = t3 ∪ comm_monoid 

20-element Vector{AbstractRule}:
 ~a + 0 --> ~a
 ~a + ~b --> ~b + ~a
 ~a + inv(~a) --> 0
 ~a + (~b + ~c) --> (~a + ~b) + ~c
 ~a * (~b + ~c) --> ~a * ~b + ~a * ~c
 ~a * ~b + ~a * ~c --> ~a * (~b + ~c)
 ~a * ~a --> (~a) ^ 2
 ~a --> (~a) ^ 1
 (~a) ^ ~b * (~a) ^ ~c --> (~a) ^ (~b + ~c)
 log((~a) ^ ~b) --> ~b * log(~a)
 log(~a * ~b) --> log(~a) + log(~b)
 log(1) --> 0
 log(e) --> 1
 e ^ log(~a) --> ~a
 ~(a::Number) + ~(b::Number) => a + b
 ~(a::Number) * ~(b::Number) => a * b
 ~a * ~b --> ~b * ~a
 (~a * ~b) * ~c --> ~a * (~b * ~c)
 ~a * (~b * ~c) --> (~a * ~b) * ~c
 1 * ~a --> ~a

In [188]:
@areequal t3 (x+y)*(a+b) ((a*(x+y))+b*(x+y)) ((x*(a+b))+y*(a+b)) 

true

In [160]:
expr = :((log(e) * log(e)) * (log(a^3 * a^2)))

:((log(e) * log(e)) * log(a ^ 3 * a ^ 2))

In [161]:
g = EGraph(expr)

EGraph(IntDisjointSet([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], Base.RefValue{Bool}(true)), Dict{Int64, EClass}(5 => EClass 5 ([3], NamedTuple()), 8 => EClass 8 ([ENode(call, ^, Expr, [4, 7])], NamedTuple()), 1 => EClass 1 ([e], NamedTuple()), 6 => EClass 6 ([ENode(call, ^, Expr, [4, 5])], NamedTuple()), 11 => EClass 11 ([ENode(call, *, Expr, [3, 10])], NamedTuple()), 9 => EClass 9 ([ENode(call, *, Expr, [6, 8])], NamedTuple()), 3 => EClass 3 ([ENode(call, *, Expr, [2, 2])], NamedTuple()), 7 => EClass 7 ([2], NamedTuple()), 4 => EClass 4 ([a], NamedTuple()), 2 => EClass 2 ([ENode(call, log, Expr, [1])], NamedTuple())…), Dict{AbstractENode, Int64}(ENode(call, log, Expr, [9]) => 10, ENode(call, ^, Expr, [4, 5]) => 6, e => 1, 3 => 5, 2 => 7, a => 4, ENode(call, ^, Expr, [4, 7]) => 8, ENode(call, *, Expr, [6, 8]) => 9, ENode(call, *, Expr, [2, 2]) => 3, ENode(call, log, Expr, [1]) => 2…), Int64[], 11, Dict{Union{Function, Symbol}, Union{Function, Symbol}}(), Dict{Any, Vector{Int64}}(:l

In [162]:
Metatheory.resetbuffers!(Metatheory.DEFAULT_BUFFER_SIZE)

CircularDeque{Tuple{Int64, Int64}}([])

In [163]:
saturate!(g, t3)

SaturationReport
	Stop Reason: timeout
	Iterations: 8
	EGraph Size: 203 eclasses, 7229 nodes
 ────────────────────────────────────────────────────────────────────
                            Time                    Allocations      
                   ───────────────────────   ────────────────────────
 Tot / % measured:      158ms /  99.8%           27.7MiB / 100.0%    

 Section   ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────
 Apply          8   88.2ms   55.9%  11.0ms   13.4MiB   48.5%  1.68MiB
 Rebuild        8   57.5ms   36.4%  7.19ms   8.11MiB   29.3%  1.01MiB
 Search         8   12.0ms    7.6%  1.50ms   6.14MiB   22.2%   786KiB
   9            8   2.32ms    1.5%   290μs   1.48MiB    5.3%   189KiB
   6            8   1.69ms    1.1%   211μs   1.25MiB    4.5%   159KiB
   4            8   1.37ms    0.9%   172μs   1.04MiB    3.8%   133KiB
   12           8    862μs    0.5%   108μs    131KiB    0.5%  16.4

In [164]:
t4  = @theory a b c begin 
    a * (b * c) == (a * b) * c
    a + (b + c) == (a + b) + c
    a * b == b * a
    a + b == b + a
    a * (b + c) == (a * b) + (a * c)
end

5-element Vector{EqualityRule}:
 ~a * (~b * ~c) == (~a * ~b) * ~c
 ~a + (~b + ~c) == (~a + ~b) + ~c
 ~a * ~b == ~b * ~a
 ~a + ~b == ~b + ~a
 ~a * (~b + ~c) == ~a * ~b + ~a * ~c

In [165]:
using Metatheory.EGraphs

In [202]:
Pkg.add("TermInterface")

   Resolving package versions...


    Updating `/workspaces/julia-playground/GARules.jl/Project.toml`
  [8ea1fca8] + TermInterface v0.3.3
  No Changes to `/workspaces/julia-playground/GARules.jl/Manifest.toml`


In [203]:
using TermInterface

In [210]:
exprhead

exprhead (generic function with 4 methods)

In [233]:
function EGraphs.make(::Val{:OddEvenAnalysis}, g::EGraph, n::ENodeLiteral)
    if n.value isa Integer
        return iseven(n.value) ? :even : :odd
    else 
        return nothing
    end
end

function EGraphs.make(::Val{:OddEvenAnalysis}, g::EGraph, n::ENodeTerm)
    println(n)
    # Let's consider only binary function call terms.
    if exprhead(n) == :call && arity(n) == 2
        op = operation(n)
        # Get the left and right child eclasses
        child_eclasses = arguments(n)
        l = g[child_eclasses[1]]
        r = g[child_eclasses[2]]

        # Get the corresponding OddEvenAnalysis value of the children
        # defaulting to nothing 
        ldata = getdata(l, :OddEvenAnalysis, nothing)
        rdata = getdata(r, :OddEvenAnalysis, nothing)

        if ldata isa Symbol && rdata isa Symbol
            if op == :*
                if ldata == rdata
                    ldata
                elseif (ldata == :even || rdata == :even) 
                    :even
                else
                    println("EGraphs.make: ldata, rdata: an expression cannot be odd and even at the same time!")
                    nothing 
                end
            elseif op == :+
                (ldata == rdata) ? :even : :odd
            end
        elseif isnothing(ldata) && rdata isa Symbol && op == :*
            rdata
        elseif ldata isa Symbol && isnothing(rdata) && op == :*
            ldata
        end
    end

    println("EGraphs.make: an expression cannot be odd and even at the same time!")
    return nothing
end

function EGraphs.join(::Val{:OddEvenAnalysis}, a, b)
    if a == b 
        return a 
    else
        # an expression cannot be odd and even at the same time!
        # this is contradictory, so we ignore the analysis value
        println("EGraphs.join: an expression cannot be odd and even at the same time!")
        return nothing 
    end
end

In [234]:
function custom_analysis(expr)
    g = EGraph(expr)
    saturate!(g, t4)
    analyze!(g, :OddEvenAnalysis)
    return getdata(g[g.root], :OddEvenAnalysis)
end

custom_analysis (generic function with 1 method)

In [235]:
custom_analysis(:(2*a)) # :even

ENode(call, *, Expr, [2, 1])
EGraphs.make: an expression cannot be odd and even at the same time!
ENode(call, *, Expr, [1, 2])
EGraphs.make: an expression cannot be odd and even at the same time!
ENode(call, *, Expr, [2, 1])
EGraphs.make: an expression cannot be odd and even at the same time!
ENode(call, *, Expr, [1, 2])
EGraphs.make: an expression cannot be odd and even at the same time!
